# QA 데이터셋 만들기 

In [3]:
from typing import List
from pydantic import BaseModel, Field, HttpUrl

class Page(BaseModel):
    id: str = Field(..., description="ID of the Page")
    link: HttpUrl = Field(description="Url link of the page")
    name: str = Field(description="Name of the page")
    parent: str = Field(default="", description="ID of the parent page")
    child: List[str] = Field(default=[], description="List of ids of the child pages")
    description: str = Field(default="", description="Description of the page")
    description_clean: str = Field(default="", description="Content markdown")
    html_content: str = Field(default="", description="HTML code of the main content in the page")

    def __hash__(self):
        return hash((self.link, self.name))

    def __eq__(self, other):
        if not isinstance(other, Page):
            return False
        return (self.link, self.name) == (other.link, other.name)

## DB 가지고 오기 

In [4]:
import json

data_dir = '/Users/jwlee-pro/Documents/Workspace_2025/projects/llm-rag-chatbot/data/db-rngd_sdk_v2.json'

with open(data_dir, "r") as f:
    data = json.load(f)
    final_pages = [Page.model_validate_json(page) for page in data["sdk"]]


In [5]:
final_pages[0]

Page(id='3e9b309f-d9d2-4ee7-be9e-9ffe421d4cfa', link=HttpUrl('https://furiosa-ai.github.io/docs-dev/2024.1/en/furiosa_llm/intro.html'), name='intro', parent='', child=[], description='\n\n\n\n\n* [.rst](../_sources/furiosa_llm/intro.rst "Download source file")\n* .pdf\n\n\n\n\n\n\n\n\n\n\nFuriosa LLM\n===========\n\n\n\n\n\nFuriosa LLM\n[#](#furiosa-llm "Link to this heading")\n====================================================\n\nFuriosa LLM provides a high-performance inference engine for LLM models and Multi-Modal LLM models,\nFuriosa LLM is designed to provide the state-of-the-art serving optimization.\nThe features of Furiosa LLM includes:\n\n* vLLM-compatible API\n* Efficient KV cache management with PagedAttention\n* Continuous batching of incoming requests in serving\n* Quantization: INT4, INT8, FP8, GPTQ, AWQ\n* Data Parallelism and Pipeline Parallelism across multiple NPUs\n* Tensor Parallelism (planned in release 2024.2) across multiple NPUs\n* OpenAI-compatible API server

In [25]:
# final_pages의 첫 번째 객체를 출력
print(final_pages[0])

# final_pages 전체 크기 확인
print(f"Total pages loaded: {len(final_pages)}")

id='3e9b309f-d9d2-4ee7-be9e-9ffe421d4cfa' link=HttpUrl('https://furiosa-ai.github.io/docs-dev/2024.1/en/furiosa_llm/intro.html') name='intro' parent='' child=[] description='\n\n\n\n\n* [.rst](../_sources/furiosa_llm/intro.rst "Download source file")\n* .pdf\n\n\n\n\n\n\n\n\n\n\nFuriosa LLM\n===========\n\n\n\n\n\nFuriosa LLM\n[#](#furiosa-llm "Link to this heading")\n====================================================\n\nFuriosa LLM provides a high-performance inference engine for LLM models and Multi-Modal LLM models,\nFuriosa LLM is designed to provide the state-of-the-art serving optimization.\nThe features of Furiosa LLM includes:\n\n* vLLM-compatible API\n* Efficient KV cache management with PagedAttention\n* Continuous batching of incoming requests in serving\n* Quantization: INT4, INT8, FP8, GPTQ, AWQ\n* Data Parallelism and Pipeline Parallelism across multiple NPUs\n* Tensor Parallelism (planned in release 2024.2) across multiple NPUs\n* OpenAI-compatible API server\n* Variou

## QA 데이터셋 만들기 

In [17]:
import os

# openAI key 
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

version #1

In [60]:
from markdownify import markdownify as md
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
import pandas as pd

class QAPair(BaseModel):
    question: str = Field(description="Question generated by llm")
    answer: str = Field(description="Answer generated by llm")

output_parser = PydanticOutputParser(pydantic_object=QAPair)


# LLM 및 Prompt 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

prompt = PromptTemplate(
    template="""As an expert in creating educational Question-Answer datasets, your task is to generate one high-quality QA pair based on the provided markdown content. 

Instructions:
1. Carefully analyze the markdown content in CONTENT section, identifying key concepts, details, and information.
2. Imagine you are a first-time visitor to a website and aim to create a challenging, abstract question that encourages deep engagement with the content.
3. Ensure that the question is specific enough that it can only be answered by referencing the given markdown.
4. Generate a concise, direct answer without introductory phrases like "The markdown says" or "Here is...".
5. Output **only one** QA pair.
6. {instructions}

Desired Format:
- Question: [Your abstract, content-specific question]
- Answer: [Your precise, context-reliant answer]

CONTENT:
{content}
""",
    partial_variables={"instructions": output_parser.get_format_instructions()},
    input_variables=["content"],
)

chain = prompt | llm | output_parser

def generate_qa(pages, qa_per_page=10):
    """
    Generate QA pairs for each page and save them into a DataFrame.

    Args:
        pages (List[Page]): List of Page objects.
        qa_per_page (int): Number of QA pairs to generate per page.

    Returns:
        pd.DataFrame: DataFrame containing generated QA pairs.
    """
    data = []

    for page in pages:
        markdown_content = md(page.html_content, strip=["img"])  # Convert HTML to markdown, stripping <img> tags

        # Generate QA pairs for each page
        for _ in range(qa_per_page):
            try:
                qa_pair = chain.invoke({"content": markdown_content})
                data.append({
                    "page_id": page.id,
                    "link": str(page.link),
                    "question": qa_pair.question,
                    "answer": qa_pair.answer,
                })
            except Exception as e:
                print(f"Error generating QA for page {page.id}: {e}")

    # Convert data to DataFrame
    df = pd.DataFrame(data)
    return df

version #3
- 중복 방지 
- html -> 마크다운 

In [63]:
from markdownify import markdownify as md
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
import pandas as pd


class QAPair(BaseModel):
    question: str = Field(description="Question generated by llm")
    answer: str = Field(description="Answer generated by llm")


output_parser = PydanticOutputParser(pydantic_object=QAPair)

# LLM 및 Prompt 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

prompt = PromptTemplate(template="""
As an expert in creating educational Question-Answer datasets, your task is to generate one high-quality QA pair based on the provided markdown content.

Instructions:
1. Carefully analyze the markdown content in CONTENT section, identifying key concepts, details, and information.
2. Avoid generating a question that is similar to any previously generated questions listed in PREVIOUS_QUESTIONS.
3. Imagine you are a first-time visitor to a website and aim to create a challenging, abstract question that encourages deep engagement with the content.
4. Ensure that the question is specific enough that it can only be answered by referencing the given markdown.
5. Generate a concise, direct answer without introductory phrases like "The markdown says" or "Here is...".
6. Output the result in the following JSON format:

```json
{{
    "question": "Your abstract, content-specific question",
    "answer": "Your precise, context-reliant answer"
}}
                        
CONTENT:
{content}

PREVIOUS_QUESTIONS:
{previous_questions}
""",
    input_variables=["content", "previous_questions"],
)

chain = prompt | llm | output_parser

def generate_qa(pages, qa_per_page=10):
    """
    Generate QA pairs for each page and save them into a DataFrame.
    Args:
        pages (List[Page]): List of Page objects.
        qa_per_page (int): Number of QA pairs to generate per page.

    Returns:
        pd.DataFrame: DataFrame containing generated QA pairs.
    """
    data = []

    for page in pages:
        markdown_content = md(page.html_content, strip=["img"])  # Convert HTML to markdown, stripping <img> tags
        previous_questions = set()

        # Generate QA pairs for each page
        for _ in range(qa_per_page):
            try:
                previous_questions_str = "\n".join(previous_questions) if previous_questions else "None"
                qa_pair = chain.invoke({
                    "content": markdown_content,
                    "previous_questions": previous_questions_str,
                })

                # 중복된 질문 방지
                if qa_pair.question in previous_questions:
                    print(f"Duplicate question detected and skipped: {qa_pair.question}")
                    continue

                # 데이터 추가
                data.append({
                    "page_id": page.id,
                    "link": str(page.link),
                    "question": qa_pair.question,
                    "answer": qa_pair.answer,
                })

                # 현재 질문 저장
                previous_questions.add(qa_pair.question)
            except Exception as e:
                print(f"Error generating QA for page {page.id}: {e}")

    # Convert data to DataFrame
    df = pd.DataFrame(data)
    return df


자 이제 QA 데이터셋 생성해보자. 

In [64]:
# Example Usage
qa_per_page = 3  # Number of QA pairs per page
qa_df = generate_qa(final_pages, qa_per_page)
qa_df

,page_id,link,question,answer
0,3e9b309f-d9d2-4ee7-be9e-9ffe421d4cfa,https://furiosa-ai.github.io/docs-dev/2024.1/e...,What are the planned features for Furiosa LLM'...,Planned features for Furiosa LLM include Tenso...
1,3e9b309f-d9d2-4ee7-be9e-9ffe421d4cfa,https://furiosa-ai.github.io/docs-dev/2024.1/e...,What are the key features of Furiosa LLM that ...,Furiosa LLM features include a vLLM-compatible...
2,3e9b309f-d9d2-4ee7-be9e-9ffe421d4cfa,https://furiosa-ai.github.io/docs-dev/2024.1/e...,How does Furiosa LLM manage efficient KV cache...,Furiosa LLM manages efficient KV cache through...
3,dcd59fbc-fb76-4f34-b6ec-ea88a833b047,https://furiosa-ai.github.io/docs-dev/2024.1/e...,What are the default configuration values for ...,The default configuration values for deploying...
4,dcd59fbc-fb76-4f34-b6ec-ea88a833b047,https://furiosa-ai.github.io/docs-dev/2024.1/e...,What are the functionalities provided by the F...,The Furiosa device plugin discovers Furiosa NP...
...,...,...,...,...
58,a214fb49-b797-4d38-b877-597b6bb059eb,https://furiosa-ai.github.io/docs-dev/2024.1/e...,What command can be used to verify the install...,The command 'lspci -nn | grep FuriosaAI' can b...
59,a214fb49-b797-4d38-b877-597b6bb059eb,https://furiosa-ai.github.io/docs-dev/2024.1/e...,What are the necessary steps to upgrade the fi...,"To upgrade the firmware of FuriosaAI devices, ..."
60,1ba93fae-bf2e-42c1-a66d-dabbee880912,https://furiosa-ai.github.io/docs-dev/2024.1/e...,What are the constraints when specifying NPU r...,"When specifying NPU resources, you can set NPU..."
61,1ba93fae-bf2e-42c1-a66d-dabbee880912,https://furiosa-ai.github.io/docs-dev/2024.1/e...,How does the deployment of Furiosa Feature Dis...,Furiosa Feature Discovery labels nodes based o...


In [46]:
# data_dir = '/Users/jwlee-pro/Documents/Workspace_2025/projects/llm-rag-chatbot/data'
# file_dir = f'{data_dir}/qa-rngd_sdk.csv'
# qa_df.to_csv(file_dir, encoding="utf-8")

In [47]:
# def remove_duplicates(qa_df):
#     # 중복된 질문 제거
#     qa_df = qa_df.drop_duplicates(subset=["question"], keep="first")
#     return qa_df

# remove_duplicates(qa_df)
# file_dir = f'{data_dir}/qa-rngd_sdk_v2.csv'
# qa_df.to_csv(file_dir, encoding="utf-8")

In [65]:
data_dir = '/Users/jwlee-pro/Documents/Workspace_2025/projects/llm-rag-chatbot/data'
file_dir = f'{data_dir}/qa-rngd_sdk_v3.csv'
qa_df.to_csv(file_dir, encoding="utf-8")

## Generate QA of card page with RAGAS

In [66]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "FuriosaAI RNGD Ragas QA Dataset"

In [87]:
from markdownify import markdownify as md
from llama_index.core import Document 

class CustomDocument(Document):
    page_content: str = Field(default="", description="Additional content for the document")

def convert_page_to_llama_index_document(page: Page) -> CustomDocument:
    return CustomDocument(
        doc_id=page.id,
        metadata={
            "source": str(page.link),
            "title": page.name,
            "parent_doc_id": page.parent,
            "child_doc_ids": json.dumps(page.child),
        },
        text=page.description_clean,  # 기본 text
        page_content=page.description_clean,  # 추가 속성
    )

docs = [convert_page_to_llama_index_document(page) for page in final_pages]
docs

[CustomDocument(id_='3e9b309f-d9d2-4ee7-be9e-9ffe421d4cfa', embedding=None, metadata={'source': 'https://furiosa-ai.github.io/docs-dev/2024.1/en/furiosa_llm/intro.html', 'title': 'intro', 'parent_doc_id': '', 'child_doc_ids': '[]'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='* [.rst](../_sources/furiosa_llm/intro.rst "Download source file") * .pdf\nFuriosa LLM ===========\nFuriosa LLM [#](#furiosa-llm "Link to this heading") ====================================================\nFuriosa LLM provides a high-performance inference engine for LLM models and Multi-Modal LLM models, Furiosa LLM is designed to provide the state-of-the-art serving optimization. The features of Furiosa LLM includes:\n* vLLM-compatible API * Efficient KV cache management with PagedAttention * Continuous batching of incoming requests in serving * Quantiza

In [68]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

Wrapp the LLMs in LangchainLLMWrapper so that it can be used with ragas.

In [69]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [82]:
docs[0]

Document(id_='3e9b309f-d9d2-4ee7-be9e-9ffe421d4cfa', embedding=None, metadata={'source': 'https://furiosa-ai.github.io/docs-dev/2024.1/en/furiosa_llm/intro.html', 'title': 'intro', 'parent_doc_id': '', 'child_doc_ids': '[]'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='* [.rst](../_sources/furiosa_llm/intro.rst "Download source file") * .pdf\nFuriosa LLM ===========\nFuriosa LLM [#](#furiosa-llm "Link to this heading") ====================================================\nFuriosa LLM provides a high-performance inference engine for LLM models and Multi-Modal LLM models, Furiosa LLM is designed to provide the state-of-the-art serving optimization. The features of Furiosa LLM includes:\n* vLLM-compatible API * Efficient KV cache management with PagedAttention * Continuous batching of incoming requests in serving * Quantization: I

In [93]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=50)

Applying HeadlineSplitter:   0%|          | 0/21 [00:00<?, ?it/s]           unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
Applying SummaryExtractor:  48%|████▊     | 13/27 [00:03<00:03,  4.38it/s]Property 'summary' already exists in node '8bf36e'. Skipping!
Property 'summary' already exists in node '4c3e39'. Skipping!
Applying SummaryExtractor:  59%|█████▉    | 16/27 [00:04<00:01,  5.89it/s]Property 'summary' already exists in node '0b7164'. Skipping!
Property 'summary' already exists in node '15a150'. Skipping!
Applying SummaryExtractor:  81%|████████▏ | 22/27 [00:04<00:00,  9.22it/s]Property 'summary' already exists in node 'bbe529'. Skipping!
Property 'summary' already exists in node '79cbfc'. Skipping!
Applying SummaryExtractor:

In [94]:
# save data
file_dir = f'{data_dir}/qa-rngd_sdk_ragas_v2'

dataset_df = dataset.to_pandas()
dataset_df.to_parquet(f"{file_dir}.parquet")
dataset_df.to_csv(f"{file_dir}.csv")

In [92]:
dataset_df

,user_input,reference_contexts,reference,synthesizer_name
0,What is FuriosaAI's role in running MLPerf Inf...,[* [.rst](../_sources/getting_started/furiosa_...,FuriosaAI Software Stack provides the `furiosa...,single_hop_specifc_query_synthesizer
1,Howw doo youu usee Furiosa LLM withh FuriosaAI...,[<1-hop>\n\n* [.rst](../_sources/getting_start...,To use Furiosa LLM with FuriosaAI NPU in a con...,multi_hop_abstract_query_synthesizer
2,What are the installation requirements for Fur...,[<1-hop>\n\nRunning Furiosa LLM in container e...,The installation requirements for Furiosa LLM ...,multi_hop_specific_query_synthesizer


In [10]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-4o", temperature=0.2)
critic_llm = ChatOpenAI(model="gpt-4o", temperature=0.2)
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 1,
    # multi_context: 1,
    # reasoning: 0.35,
    # conditional: 0.2,
}

testset = generator.generate_with_llamaindex_docs(docs, 10, distributions) 
testset_df = testset.to_pandas()
testset_df.to_parquet("v2/qa_ragas_2.parquet")
testset_df.to_csv("v2/qa_ragas_2.csv")

ModuleNotFoundError: No module named 'ragas.testset.evolutions'